In [ ]:
from spyral_utils.plot import Histogrammer
from spyral_utils.nuclear import NuclearDataMap
from spyral_utils.nuclear.particle_id import deserialize_particle_id
from spyral.core.constants import DEG2RAD
from spyral.core.run_stacks import form_run_string

import numpy as np
import polars as pl
import matplotlib.pyplot as plt
from pathlib import Path

%matplotlib widget

RAD2DEG = 1.0/DEG2RAD

In [ ]:
# Load config
workspace_path = Path("/Volumes/e20009/e20009_analysis")
estimation_result_path = workspace_path / "Estimation"
pid_path = Path('/Users/attpc/Desktop/e20009_analysis/e20009_analysis/e20009_parameters/pid.json')

# Set the run range (inclusive)
run_min = 347
run_max = 348

# IC gate
ic_min_val = 0.0
ic_max_val = 4095.0

# Nucleus map
nuclear_map = NuclearDataMap()

In [ ]:
# Make utility objects for plotting and making cuts
grammer = Histogrammer()

In [ ]:
# Create histogram
grammer.add_hist2d("particle_id", (400, 400), ((-100.0, 10e3), (-0.1, 2.5))) # Plot of dEdx vs. Brho (particle ID)

In [ ]:
# Fill histograms
for run in range(run_min, run_max+1):
    run_path = estimation_result_path / f"{form_run_string(run)}.parquet"
    if not run_path.exists():
        continue
    df = pl.read_parquet(run_path)
    # The below filter is optional. Filter the data on the ion chamber gate. Comment/Uncomment the line below to turn on/off the filter
    df = df.filter((pl.col('ic_amplitude') > ic_min_val) & (pl.col('ic_amplitude') < ic_max_val))
    grammer.fill_hist2d('particle_id', df.select('dEdx').to_numpy(), df.select('brho').to_numpy())

In [ ]:
# Load PID vertices
pid = deserialize_particle_id(pid_path, nuclear_map)
vertices = np.asarray(pid.cut.get_vertices())

In [ ]:
# Find gain factor
gain_factor = 1

pid_hist = grammer.get_hist2d("particle_id")
fig, ax = plt.subplots(1,1)
mesh = ax.pcolormesh(pid_hist.x_bins, pid_hist.y_bins, pid_hist.counts, norm='log')
fig.colorbar(mesh, ax=ax)
ax.plot(vertices[:, 0] / gain_factor, vertices[:, 1], color = 'red')
ax.set_title("Particle ID")
ax.set_xlabel("dE/dx(arb.)")
ax.set_ylabel("B$\\rho$(Tm)")
fig.set_figheight(8.0)
fig.set_figwidth(11.0)